<a href="https://colab.research.google.com/github/asutosh07/Bitcoin-Stock-Prediction-Using-LSTMs/blob/master/BitCoinPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

In [2]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.DataFrame()

files = ["BTC-USD","ETH-USD","LTC-USD","BCH-USD"]

for i in files:
  file_path = f'/content/drive/My Drive/{i}.csv'
  temp_file = pd.read_csv(file_path,names=['timestamp', 'low', 'high', 'open', 'close', 'volume'])
  
  temp_file.rename(columns={"close": f"{i}_close", "volume": f"{i}_volume"}, inplace=True)
  temp_file.set_index("timestamp",inplace=True)
  temp_file = temp_file[[f"{i}_close",f"{i}_volume"]]
  if len(df)==0:
    df = temp_file
  else:
    df = df.join(temp_file)

df.fillna(method="ffill",inplace=True)
df.dropna(inplace = True)
print(df.head())

In [5]:
prev_seq_len = 60
futurePeriodOfPrediction = 3
coin_pred = "BTC-USD" 

In [6]:
def targetValue(current_val,future_val):
  if float(future_val)>float(current_val):
    return 1
  return 0

In [7]:
df['target'] = list(map(targetValue,df[f'{coin_pred}_close'],df[f'{coin_pred}_close'].shift(-futurePeriodOfPrediction)))
#df = df.replace([np.inf, -np.inf], np.nan)

In [ ]:
print(df.head())

In [ ]:
sorted_timestamp = sorted(df.index.values)
val_time_edge = sorted(df.index.values)[-int(0.05*len(sorted_timestamp))]

validation_set = df[(df.index>=val_time_edge)]
train = df[(df.index<val_time_edge)]

print(train.head())
print(validation_set.head())

In [10]:
from sklearn import preprocessing
from collections import deque
import random
import numpy as np

In [11]:
def preprocessing_df(df):

  df = df.replace([np.inf, -np.inf], np.nan)
  for col in df.columns:

    if col!="target":
      df[col] = df[col].pct_change()
      
      df.dropna(inplace=True)
      #print(df)
      df[col] = preprocessing.scale(df[col].values)
      #print(df[col])

  df.dropna(inplace=True)

  sequential_data = []
  prev_days = deque(maxlen =prev_seq_len )

  for i in df.values:
    prev_days.append([j for j in i[::-1]])
    if len(prev_days) == prev_seq_len:
      sequential_data.append([np.array(prev_days),i[-1]])
  random.shuffle(sequential_data)

  buys = []
  sells = []

  for seq,target in sequential_data:
    if target == 0:
      sells.append([seq,target])
    elif target==1:
      buys.append([seq,target])
  
  random.shuffle(buys)
  random.shuffle(sells)

  lower = min(len(buys),len(sells))
  #print(lower)
  buys = buys[:lower]
  sells = sells[:lower]

  sequential = buys+sells
  random.shuffle(sequential_data)
  print("Buys , Sells: ",len(buys),len(sells))
  x_set =[]
  y_set =[]

  for seq,target in sequential_data:
    x_set.append(seq)
    y_set.append(target)

  return np.array(x_set),y_set



In [ ]:
train_x,train_y = preprocessing_df(train)
print("Size of training set",len(train_x))
val_x , val_y = preprocessing_df(validation_set)
print("Size of validation set",len(val_x))

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization


In [ ]:
model = Sequential()
model.add(LSTM(128,input_shape=(train_x.shape[1:]),activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128,activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128,activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2,activation='softmax'))




In [ ]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)

optimiser = tf.keras.optimizers.Adam()
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=optimiser,
    metrics = ['accuracy']
)
model.fit(
    train_x,train_y,
    batch_size = 32,
    epochs = 40,
    validation_data = (val_x,val_y)
)

score = model.evaluate(val_x,val_y,verbose = 0)
print('Test loss:', score[0])
print('Test accuracy: ',score[1])